In [4]:
import sys, os, shutil
import argparse
from utils import load_img_to_array, save_array_to_img, dilate_mask, \
    show_mask, show_points, get_clicked_point
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import cv2
import albumentations as A

In [5]:
def draw_bounding_boxes(image, bbox_data):
    for data in bbox_data:
        class_id, x_center, y_center, width, height = data
        x_center, y_center, width, height = [int(float(val) * image.shape[1]) if i % 2 == 0 else int(float(val) * image.shape[0]) for i, val in enumerate([x_center, y_center, width, height])]

        # 좌표 계산
        top_left = (x_center - width // 2, y_center - height // 2)
        bottom_right = (x_center + width // 2, y_center + height // 2)

        # 바운딩 박스 그리기
        cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
        # 레이블 표시
        cv2.putText(image, str(class_id), (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [6]:
def draw_three_images(image1, image2, image3, bbox_data):
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 20))
    ax1.imshow(image1)
    ax1.axis('off')

    # 바운딩 박스와 레이블을 그린 이미지
    #draw_bounding_boxes(image2, bbox_data)
    ax2.imshow(image2)
    ax2.axis('off')

    draw_bounding_boxes(image3, bbox_data)
    ax3.imshow(image3)
    ax3.axis('off')
    plt.show()

In [7]:
def apply_transform_to_masked_area(image, mask, transform):
    # 마스크 영역에만 변형을 적용하기 위해 ROI를 추출합니다.
    roi = image.copy()
    roi[mask == 0] = 0  # 마스크가 0인 부분을 제거 (검은색으로 만듦)

    # 변형 적용
    transformed_img = transform(image=roi)['image']
    # 변형된 이미지를 원본 이미지에 다시 합치기
    image[mask != 0] = transformed_img[mask != 0]

    return image

def apply_transform_to_background(image, mask, transform):
    # 배경에만 변형을 적용하기 위해 배경 ROI를 추출합니다.
    background_roi = image.copy()
    background_roi[mask != 0] = 0  # 마스크가 0이 아닌 부분을 제거 (검은색으로 만듦)

    # 변형 적용
    transformed_background = transform(image=background_roi)['image']

    # 변형된 배경을 원본 이미지에 다시 합치기
    # 이 때 객체가 있는 부분은 그대로 유지합니다.
    image[mask == 0] = transformed_background[mask == 0]

    return image

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
        


In [8]:
img_folder_path =f'../wim_data/final_dataset/train/images/'
mask_folder_path =f'../wim_data/object_processing_masks/'
txt_folder_path = f'../wim_data/train/labels/'

mask_file = '2023-04-24_41.mp4#t=126_mask.jpg'

### parameters

In [9]:
# task_name = 'bg': 배경 변환, 'ob': 오브젝트 변환
task_name = 'ob'
brightness_from, brightness_to = 0.5, 1
contrast_from, contrast_to = 0, 0

transform = A.Compose([
    A.RandomBrightnessContrast(
        always_apply=True, p=1.0,
        brightness_limit=(brightness_from,brightness_to),
        contrast_limit=(contrast_from, contrast_to),
        brightness_by_max=False),
])
# Declare an augmentation pipeline

#### 자동화

In [10]:
mask_folder_path =f'../wim_data/object_processing_masks/'
img_folder_path =f'../wim_data/final_dataset/train/images/'

In [11]:
mask_files = [f for f in os.listdir(mask_folder_path) if f.endswith('.jpg') or f.endswith('.png')]
print(len(mask_files))

1200


In [14]:
par_name = f'{task_name}_bright_{brightness_from}-{brightness_to}_contrast_{contrast_from}-{contrast_to}'
processed_image_path = f'../wim_data/processed_img_final/{par_name}/'
train_label_path = '/raid/coss07/yolov8/yolov8/Dataset_bind_16_17_21_24_25_26_27_29_31_32_7_20_28_deleted/train/labels'
val_path = '/raid/coss07/yolov8/yolov8/Dataset_bind_16_17_21_24_25_26_27_29_31_32_7_20_28_deleted/val'
test_path = '/raid/coss07/yolov8/yolov8/Dataset_bind_16_17_21_24_25_26_27_29_31_32_7_20_28_deleted/test'

In [15]:
for mask_file in tqdm(mask_files):
    splited = os.path.splitext(mask_file)
    image_file = splited[0][:-5] + splited[1]
    img = load_img_to_array(img_folder_path+image_file)
    mask = load_img_to_array(mask_folder_path + mask_file)
    
    transformed_img = apply_transform_to_masked_area(img, mask, transform)
    
    create_folder_if_not_exists(processed_image_path+'train')
    create_folder_if_not_exists(processed_image_path+'train/images')
    
    save_array_to_img(transformed_img, processed_image_path+'train/images/' + image_file)
    
shutil.copytree(train_label_path, processed_image_path+'train/labels')
shutil.copytree(val_path, processed_image_path+'val')
shutil.copytree(test_path, processed_image_path+'test')


100%|██████████| 1200/1200 [05:18<00:00,  3.77it/s]


'../wim_data/processed_img_final/ob_bright_0.5-1_contrast_0-0/test'

In [ ]:
splited = os.path.splitext(mask_file)
image_file = splited[0][:-5] + splited[1]

print(img_folder_path+image_file)
img2 = load_img_to_array(img_folder_path+image_file)
img = load_img_to_array(img_folder_path+image_file)
mask = load_img_to_array(mask_folder_path + mask_file)


txt_file = os.path.join(txt_folder_path, os.path.splitext(image_file)[0] + '.txt')

with open(txt_file, 'r') as file:
    lines = file.readlines()
    bbox_data = [line.strip().split() for line in lines]

# 기호에 맞게 ㅎ
#transformed_img = apply_transform_to_background(img2, mask, transform)
transformed_img = apply_transform_to_masked_area(img2, mask, transform)

draw_three_images(img,mask, img2, bbox_data)

In [49]:
shutil.copytree(val_path, processed_image_path+'val')
#shutil.copytree(test_path, processed_image_path+'test')

FileExistsError: [Errno 17] File exists: '../wim_data/processed_images/bg_bright_0.5-1_contrast_0-0/val'